In [25]:
from os.path import exists
import investpy
import pandas as pd
import os
import yfinance as yf
from functools import reduce

from_date = '04/01/1999'
to_date = '04/01/2022'

In [29]:
# Get list of stocks

au_stocks = ['tls', 'bhp', 'anz', 'cba', 'csl', 'wow', 'fmg', 'mqg', 'brg', 'cpu', 'gpt', 'qbe', 'sun', 'wes']

au_stocks = [i+'.ax' for i in au_stocks]

In [33]:
au_stocks = ['rio', 'nab', 'wbc', 'fmg', 'tcl', 'all', 'wpl', 'sto', 'ncm', 'rea', 'shl', 'rhc', 'bxb', 'coh']

au_stocks = [i+'.ax' for i in au_stocks]

In [34]:
# Save data for each stock

for fund in au_stocks:
    df = yf.Ticker(fund).history(period='max')
    df.to_csv(f'./rawStocks/{fund}.csv', index='Date')

In [40]:
# Get all CSV's

def find_csvs(path, suffix='.csv'):
    files = os.listdir(path)
    return [file for file in files if file.endswith(suffix)]
files = find_csvs('rawStocks')

useable_files = []

for f in files:
    df = pd.read_csv(f'./rawStocks/{f}')
    useable_files.append(f)

In [41]:
# Take close price from each df

dataframes = []

for file in useable_files:

    df = pd.read_csv(f'./rawStocks/{file}')
    new_df = df[['Date', 'Close']]
    new_df.columns = ['Date', str(file)[:-4]]
    dataframes.append(new_df)

In [43]:
# Combine and save dataframes

au_stocks_df = reduce(lambda x, y: pd.merge(x, y, on='Date', how='outer', sort=True), dataframes)
au_stocks_df.to_csv('au_stocks.csv', index=False)

In [41]:
# Merge files
for dataframe, idx in dataframes:
    df = dataframes[0].merge(dataframes[1], on='Date', how='outer')

# Sort df
df.sort_values('Date', inplace=True)

# Fill Na
df.fillna(inplace=True, method='ffill')

# Save DF
df.to_csv('aus_stocks.csv', index=False)

[            Date  Silver
 0     1999-01-04   4.902
 1     1999-01-05   5.004
 2     1999-01-06   5.129
 3     1999-01-07   5.204
 4     1999-01-08   5.254
 ...          ...     ...
 6243  2021-12-30  23.091
 6244  2021-12-31  23.381
 6245  2022-01-02  23.328
 6246  2022-01-03  22.840
 6247  2022-01-04  23.086
 
 [6248 rows x 2 columns],
             Date    Gold
 0     1999-01-04   287.4
 1     1999-01-05   286.9
 2     1999-01-06   287.6
 3     1999-01-07   291.6
 4     1999-01-08   291.2
 ...          ...     ...
 5874  2021-12-29  1805.8
 5875  2021-12-30  1814.1
 5876  2021-12-31  1828.6
 5877  2022-01-03  1800.1
 5878  2022-01-04  1814.6
 
 [5879 rows x 2 columns],
             Date  Copper
 0     1999-01-04  0.6440
 1     1999-01-05  0.6390
 2     1999-01-06  0.6450
 3     1999-01-07  0.6450
 4     1999-01-08  0.6530
 ...          ...     ...
 5821  2021-12-30  4.3925
 5822  2021-12-31  4.4595
 5823  2022-01-02  4.4603
 5824  2022-01-03  4.4207
 5825  2022-01-04  4.4665
 
 [5826

In [ ]:
# Get available commodities saved from before

with open('./rawComs/available_commodities.txt', 'r') as f:
    avail_coms = f.readlines()
avail_coms = eval(avail_coms[0])

com_problems = []

In [ ]:
# Get commoditity data

for asset in assets:
    com_df = investpy.commodities.get_commodity_historical_data(asset, from_date='04/01/1999', to_date='04/01/2022')
    time.sleep(1)
    com_df = com_df.reset_index()
    com_df.to_csv(f'./dataframes/rawComs/{asset}.csv', index=False)

In [ ]:
assets = avail_coms

def add_commodity(df, asset):
    try:
        com_df = investpy.commodities.get_commodity_historical_data(asset, from_date='04/01/1999', to_date='04/01/2022')
        time.sleep(1)
        com_col = com_df['Close']
        com_df = com_col.reset_index()
        com_df.columns = ['Date', str(asset)]
        df['Date'] = pd.to_datetime(df['Date'])
        new_df = df.merge(com_df, on='Date', how='left')
        return new_df
    except Exception as e:
        com_problems.append(asset)
        print("Error with: ", asset, '. ', e)
        return df
        